In [1]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "petermchale/xgen-7b-tuned-alpaca"
config = PeftConfig.from_pretrained(peft_model_id)
config

PeftConfig(peft_type='LORA', auto_mapping=None, base_model_name_or_path='Salesforce/xgen-7b-8k-base', revision=None, task_type='CAUSAL_LM', inference_mode=True)

In [2]:
original_model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


In [4]:
peft_model = PeftModel.from_pretrained(original_model, peft_model_id, is_trainable=False)

In [12]:
# prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: Generate a list of ten items a person might need for a camping trip ### Response:"
# prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: What is the capital of France? ### Response:"
prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: Give three tips for staying healthy. ### Response:"

batch = tokenizer(prompt, return_tensors='pt')
input_ids = batch['input_ids']
input_ids = input_ids.to('cuda')

output_tokens_from_original_model = original_model.generate(input_ids=input_ids, max_new_tokens=50)

print(tokenizer.decode(output_tokens_from_original_model[0], skip_special_tokens=True))

Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: Give three tips for staying healthy. ### Response: 1. Eat a balanced diet with plenty of fruits, vegetables, and whole grains. 2. Exercise regularly to stay active and maintain a healthy weight. 3. Get enough sleep to keep your body and mind functioning properly.


In [13]:
output_tokens_from_peft_model = peft_model.generate(input_ids=input_ids, max_new_tokens=50)

print(tokenizer.decode(output_tokens_from_peft_model[0], skip_special_tokens=True))

Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: Give three tips for staying healthy. ### Response: 1. Eat a balanced diet with plenty of fruits, vegetables, and whole grains. 2. Exercise regularly to stay active and maintain a healthy weight. 3. Get enough sleep to keep your body and mind functioning properly.


In [7]:
# compare with fine-tuning instruction dataset: 
# https://huggingface.co/datasets/tatsu-lab/alpaca/viewer/tatsu-lab--alpaca

## Remarks 

I see no difference between the output of the original model and the fine-tuned model. Yet, the models do appear to be different: 

In [14]:
original_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(51200, 4096, padding_idx=0)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear8bitLt(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): D

In [15]:
peft_model 

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(51200, 4096, padding_idx=0)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
              (v_p